# പ്രാധാന്യമുള്ള അംഗ മിഡിൽവെയർ ഉപയോഗിച്ച് ഹോട്ടൽ ബുക്കിംഗ്

ഈ നോട്ട്ബുക്ക് **ഫംഗ്ഷൻ-അടിസ്ഥാനമാക്കിയ മിഡിൽവെയർ** മൈക്രോസോഫ്റ്റ് ഏജന്റ് ഫ്രെയിംവർക്കിൽ ഉപയോഗിക്കുന്നതിനെ വിശദീകരിക്കുന്നു. നിബന്ധനാപരമായ പ്രവാഹ ഉദാഹരണത്തെ അടിസ്ഥാനമാക്കി, പ്രാധാന്യമുള്ള അംഗങ്ങൾക്ക് പ്രത്യേക ആനുകൂല്യങ്ങൾ നൽകുന്ന ഒരു മിഡിൽവെയർ ലെയർ ചേർക്കുന്നു.

## നിങ്ങൾ പഠിക്കുന്നതെന്താണ്:
1. **ഫംഗ്ഷൻ-അടിസ്ഥാനമാക്കിയ മിഡിൽവെയർ**: ഫംഗ്ഷൻ ഫലങ്ങൾ തടയുകയും മാറ്റുകയും ചെയ്യുക
2. **കോണ്റ്റെക്സ്റ്റ് ആക്സസ്**: `context.result` പ്രവർത്തനാനന്തരത്തിൽ വായിക്കുകയും മാറ്റുകയും ചെയ്യുക
3. **ബിസിനസ് ലജിക് നടപ്പാക്കൽ**: പ്രാധാന്യമുള്ള അംഗ ആനുകൂല്യങ്ങൾ
4. **ഫല ഓവർറൈഡ്**: ഉപയോക്തൃ നിലയെ അടിസ്ഥാനമാക്കി ഫംഗ്ഷൻ ഫലങ്ങൾ മാറ്റുക
5. **ഒരേ പ്രവാഹം, വ്യത്യസ്ത ഫലങ്ങൾ**: മിഡിൽവെയർ-പ്രേരിത പെരുമാറ്റ മാറ്റങ്ങൾ

## മിഡിൽവെയർ ഉപയോഗിച്ച പ്രവാഹ ആർക്കിടെക്ചർ:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## നിബന്ധനാപരമായ പ്രവാഹത്തിൽ നിന്ന് പ്രധാന വ്യത്യാസം:

**മിഡിൽവെയർ ഇല്ലാതെ** (14-conditional-workflow.ipynb):
- പാരിസിൽ മുറികൾ ഇല്ല → alternative_agent-ലേക്ക് റൂട്ടുചെയ്യുക

**മിഡിൽവെയർ ഉപയോഗിച്ച്** (ഈ നോട്ട്ബുക്ക്):
- സാധാരണ ഉപയോക്താവ് + പാരിസ് → മുറികൾ ഇല്ല → alternative_agent-ലേക്ക് റൂട്ടുചെയ്യുക
- പ്രാധാന്യമുള്ള ഉപയോക്താവ് + പാരിസ് → 🌟 മിഡിൽവെയർ ഓവർറൈഡ്! → ലഭ്യമാണ് → booking_agent-ലേക്ക് റൂട്ടുചെയ്യുക

## മുൻ‌വശങ്ങൾ:
- മൈക്രോസോഫ്റ്റ് ഏജന്റ് ഫ്രെയിംവർക്കിന്റെ ഇൻസ്റ്റാളേഷൻ
- നിബന്ധനാപരമായ പ്രവാഹങ്ങളുടെ അടിസ്ഥാന ധാരണ (14-conditional-workflow.ipynb കാണുക)
- GitHub ടോക്കൺ അല്ലെങ്കിൽ OpenAI API കീ
- മിഡിൽവെയർ പാറ്റേണുകളുടെ അടിസ്ഥാന ധാരണ


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ഘട്ടം 1: ഘടനാപരമായ ഔട്ട്പുട്ടുകൾക്കുള്ള Pydantic മോഡലുകൾ നിർവചിക്കുക

ഈ മോഡലുകൾ ഏജന്റുകൾ തിരികെ നൽകുന്ന **സ്കീമ** നിർവചിക്കുന്നു. മിഡിൽവെയർ ലഭ്യതാ ഫലത്തിൽ മാറ്റം വരുത്തുമ്പോൾ ട്രാക്ക് ചെയ്യാൻ `priority_override` ഫീൽഡ് ചേർത്തിട്ടുണ്ട്.


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ഘട്ടം 2: പ്രാധാന്യ അംഗങ്ങളുടെ ഡാറ്റാബേസ് നിർവചിക്കുക

ഈ ഡെമോയിൽ, നാം ഒരു പ്രാധാന്യ അംഗത്വ ഡാറ്റാബേസ് സിമുലേറ്റ് ചെയ്യും. പ്രൊഡക്ഷനിൽ, ഇത് ഒരു യഥാർത്ഥ ഡാറ്റാബേസ് അല്ലെങ്കിൽ API-യുമായി ക്വറി ചെയ്യും.

**പ്രാധാന്യ അംഗങ്ങൾ:**
- `alice@example.com` - VIP അംഗം
- `bob@example.com` - പ്രീമിയം അംഗം  
- `priority_user` - ടെസ്റ്റ് അക്കൗണ്ട്


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## ഘട്ടം 3: ഹോട്ടൽ ബുക്കിംഗ് ടൂൾ സൃഷ്ടിക്കുക

കണ്ടീഷണൽ വർക്‌ഫ്ലോ പോലെ തന്നെയാണ്, പക്ഷേ ഇപ്പോൾ ഇത് മിഡിൽവെയർ വഴി തടയപ്പെടും!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ഘട്ടം 4: 🌟 പ്രാധാന്യ പരിശോധന മിഡിൽവെയർ സൃഷ്ടിക്കുക (പ്രധാന സവിശേഷത!)

ഇത് ഈ നോട്ട്‌ബുക്കിന്റെ **പ്രധാന പ്രവർത്തനക്ഷമത** ആണ്. മിഡിൽവെയർ:

1. **തടയുന്നു** hotel_booking ഫംഗ്ഷൻ കോൾ
2. `next(context)` വിളിച്ച് ഫംഗ്ഷൻ സാധാരണയായി **നടത്തുന്നു**
3. `context.result` ൽ ഫലത്തെ **പരിശോധിക്കുന്നു**
4. ഉപയോക്താവ് പ്രാധാന്യമുള്ളവനാണെങ്കിൽ, മുറികൾ ലഭ്യമല്ലെങ്കിൽ ഫലം **മാറ്റുന്നു**
5. **മാറ്റിയ ഫലം** ഏജന്റിലേക്ക് തിരികെ **മടക്കുന്നു**

**പ്രധാന മാതൃക:**
```python
async def my_middleware(context, next):
    await next(context)  # ഫംഗ്ഷൻ പ്രവർത്തിപ്പിക്കുക
    # ഇപ്പോൾ context.result ഫംഗ്ഷന്റെ ഔട്ട്പുട്ട് ഉൾക്കൊള്ളുന്നു
    if some_condition:
        context.result = new_value  # ഓവർറൈഡ്!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## ഘട്ടം 5: റൂട്ടിംഗിനുള്ള കണ്ടീഷൻ ഫംഗ്ഷനുകൾ നിർവചിക്കുക

കണ്ടീഷണൽ വർക്ക്‌ഫ്ലോയിൽ ഉപയോഗിക്കുന്നതുപോലെ തന്നെ കണ്ടീഷൻ ഫംഗ്ഷനുകൾ - റൂട്ടിംഗിനായി ഘടനയുള്ള ഔട്ട്പുട്ട് പരിശോധിക്കുന്നു.


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## ഘട്ടം 6: കസ്റ്റം ഡിസ്പ്ലേ എക്സിക്യൂട്ടർ സൃഷ്ടിക്കുക

മുൻപ് ചെയ്തതുപോലെ തന്നെ എക്സിക്യൂട്ടർ - അന്തിമ വർക്ക്‌ഫ്ലോ ഔട്ട്പുട്ട് പ്രദർശിപ്പിക്കുന്നു.


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## ഘട്ടം 7: പരിസ്ഥിതി വേരിയബിളുകൾ ലോഡ് ചെയ്യുക

LLM ക്ലയന്റ് (GitHub Models അല്ലെങ്കിൽ OpenAI) ക്രമീകരിക്കുക.


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## ഘട്ടം 8: മിഡിൽവെയർ ഉപയോഗിച്ച് AI ഏജന്റുമാരെ സൃഷ്ടിക്കുക

**പ്രധാന വ്യത്യാസം:** availability_agent സൃഷ്ടിക്കുമ്പോൾ, `middleware` പാരാമീറ്റർ പാസ്സ് ചെയ്യുന്നു!

ഇതാണ് priority_check_middleware ഏജന്റിന്റെ ഫംഗ്ഷൻ ഇൻവൊക്കേഷൻ പൈപ്പ്‌ലൈനിലേക്ക് ഉൾപ്പെടുത്തുന്ന രീതി.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ഘട്ടം 9: പ്രവാഹം നിർമ്മിക്കുക

മുൻപത്തെ പോലെ തന്നെ പ്രവാഹത്തിന്റെ ഘടന - ലഭ്യതയെ അടിസ്ഥാനമാക്കിയുള്ള ഷരത്ഭദ്രമായ റൂട്ടിംഗ്.


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## ഘട്ടം 10: ടെസ്റ്റ് കേസ് 1 - സാധാരണ ഉപയോക്താവ് പാരിസിൽ (ഓവർറൈഡ് ഇല്ല)

ഒരു സാധാരണ ഉപയോക്താവ് പാരിസിൽ ബുക്ക് ചെയ്യാൻ ശ്രമിക്കുന്നു → മുറികൾ ഇല്ല → alternative_agent-ലേക്ക് റൂട്ടുചെയ്യുന്നു


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## ഘട്ടം 11: ടെസ്റ്റ് കേസ് 2 - 🌟 പ്രാധാന്യപ്പെട്ട ഉപയോക്താവ് പാരിസിൽ (ഓവർറൈഡ് സഹിതം!)

ഒരു പ്രാധാന്യപ്പെട്ട അംഗം പാരിസിൽ ബുക്ക് ചെയ്യാൻ ശ്രമിക്കുന്നു → ആദ്യം മുറികൾ ഇല്ല → 🌟 മിഡിൽവെയർ ഓവർറൈഡ് ചെയ്യുന്നു! → booking_agent-ലേക്ക് റൂട്ടുചെയ്യുന്നു

**ഇതാണ് മിഡിൽവെയറിന്റെ ശക്തിയുടെ പ്രധാന പ്രദർശനം!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## ഘട്ടം 12: ടെസ്റ്റ് കേസ് 3 - സ്റ്റോക്ക്ഹോം പ്രാധാന്യ ഉപയോക്താവ് (ഇതിനകം ലഭ്യമാണ്)

പ്രാധാന്യ ഉപയോക്താവ് സ്റ്റോക്ക്ഹോം ശ്രമിക്കുന്നു → മുറികൾ ലഭ്യമാണ് → ഓവർറൈഡ് ആവശ്യമില്ല → booking_agent-ലേക്ക് റൂട്ടുചെയ്യുന്നു

ഇത് മിഡിൽവെയർ ആവശ്യമായപ്പോൾ മാത്രമേ പ്രവർത്തിക്കുന്നുള്ളൂ എന്ന് കാണിക്കുന്നു!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## പ്രധാന ആശയങ്ങൾയും മിഡിൽവെയർ ആശയങ്ങളും

### ✅ നിങ്ങൾ പഠിച്ചതെന്ത്:

#### **1. ഫംഗ്ഷൻ-അടിസ്ഥാനമാക്കിയ മിഡിൽവെയർ മാതൃക**

മിഡിൽവെയർ ഒരു ലളിതമായ async ഫംഗ്ഷൻ ഉപയോഗിച്ച് ഫംഗ്ഷൻ കോൾ തടയുന്നു:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # ഫംഗ്ഷൻ പ്രവർത്തനത്തിന് മുമ്പ്
    print("Intercepting...")
    
    # ഫംഗ്ഷൻ പ്രവർത്തിപ്പിക്കുക
    await next(context)
    
    # ഫംഗ്ഷൻ പ്രവർത്തനത്തിന് ശേഷം - ഫലത്തെ പരിശോധിക്കുക
    if context.result:
        # ആവശ്യമെങ്കിൽ ഫലം മാറ്റം ചെയ്യുക
        context.result = modified_value
```

#### **2. കോൺടെക്സ്റ്റ് ആക്സസ് ചെയ്യലും ഫലത്തെ മാറ്റുകയും ചെയ്യുക**

- `context.function` - വിളിക്കപ്പെടുന്ന ഫംഗ്ഷൻ ആക്സസ് ചെയ്യുക
- `context.arguments` - ഫംഗ്ഷൻ ആർഗ്യുമെന്റുകൾ വായിക്കുക
- `context.kwargs` - അധിക പാരാമീറ്ററുകൾ ആക്സസ് ചെയ്യുക
- `await next(context)` - ഫംഗ്ഷൻ പ്രവർത്തിപ്പിക്കുക
- `context.result` - ഫംഗ്ഷന്റെ ഔട്ട്പുട്ട് വായിക്കുക/മാറ്റുക

#### **3. ബിസിനസ് ലജിക് നടപ്പാക്കൽ**

നമ്മുടെ മിഡിൽവെയർ പ്രാധാന്യ അംഗങ്ങളുടെ ആനുകൂല്യങ്ങൾ നടപ്പാക്കുന്നു:
- **സാധാരണ ഉപയോക്താക്കൾ**: മാറ്റങ്ങളില്ല, സാധാരണ പ്രവൃത്തി പ്രവാഹം
- **പ്രാധാന്യ ഉപയോക്താക്കൾ**: "ലഭ്യതയില്ല" → "ലഭ്യമാണ്" എന്നത് ഓവർറൈഡ് ചെയ്യുക
- **ശരിയായ സാഹചര്യത്തിൽ മാത്രം**: ആവശ്യമുള്ളപ്പോൾ മാത്രമേ ഓവർറൈഡ് ചെയ്യൂ

#### **4. ഒരേ പ്രവൃത്തി പ്രവാഹം, വ്യത്യസ്ത ഫലങ്ങൾ**

മിഡിൽവെയറിന്റെ ശക്തി:
- ✅ പ്രവൃത്തി പ്രവാഹ ഘടനയിൽ മാറ്റങ്ങളില്ല
- ✅ ടൂൾ ഫംഗ്ഷനിൽ മാറ്റങ്ങളില്ല
- ✅ കണ്ടീഷണൽ റൂട്ടിംഗ് ലജിക്കിൽ മാറ്റങ്ങളില്ല
- ✅ മിഡിൽവെയർ മാത്രം → വ്യത്യസ്ത പെരുമാറ്റം!

### 🚀 യഥാർത്ഥ ലോകത്തിലെ പ്രയോഗങ്ങൾ:

1. **VIP/പ്രീമിയം ഫീച്ചറുകൾ**
   - പ്രീമിയം ഉപയോക്താക്കൾക്കായി നിരക്ക് പരിധികൾ ഓവർറൈഡ് ചെയ്യുക
   - റിസോഴ്സുകളിലേക്ക് പ്രാധാന്യ ആക്സസ് നൽകുക
   - പ്രീമിയം ഫീച്ചറുകൾ ഡൈനാമിക്കായി അൺലോക്ക് ചെയ്യുക

2. **A/B ടെസ്റ്റിംഗ്**
   - ഉപയോക്താക്കളെ വ്യത്യസ്ത നടപ്പിലാക്കലുകളിലേക്ക് റൂട്ടുചെയ്യുക
   - പ്രത്യേക ഉപയോക്താക്കളുമായി പുതിയ ഫീച്ചറുകൾ പരീക്ഷിക്കുക
   - ഫീച്ചറുകൾ ക്രമാതീതമായി പുറത്തിറക്കുക

3. **സുരക്ഷയും അനുസരണവും**
   - ഫംഗ്ഷൻ കോൾസ് ഓഡിറ്റ് ചെയ്യുക
   - സെൻസിറ്റീവ് ഓപ്പറേഷനുകൾ തടയുക
   - ബിസിനസ് നിയമങ്ങൾ നടപ്പാക്കുക

4. **പ്രകടന മെച്ചപ്പെടുത്തൽ**
   - പ്രത്യേക ഉപയോക്താക്കൾക്കായി ഫലങ്ങൾ കാഷ് ചെയ്യുക
   - സാധ്യമായപ്പോൾ ചെലവേറിയ ഓപ്പറേഷനുകൾ ഒഴിവാക്കുക
   - ഡൈനാമിക് റിസോഴ്സ് അലോക്കേഷൻ

5. **പിശക് കൈകാര്യം ചെയ്യലും റിട്രൈയും**
   - പിശകുകൾ സുന്ദരമായി കൈകാര്യം ചെയ്യുക
   - റിട്രൈ ലജിക് നടപ്പാക്കുക
   - പകരം നടപ്പിലാക്കലിലേക്ക് ഫാൾബാക്ക് ചെയ്യുക

6. **ലോഗിംഗ് & മോണിറ്ററിംഗ്**
   - ഫംഗ്ഷൻ എക്സിക്യൂഷൻ സമയങ്ങൾ ട്രാക്ക് ചെയ്യുക
   - പാരാമീറ്ററുകളും ഫലങ്ങളും ലോഗ് ചെയ്യുക
   - ഉപയോഗ പാറ്റേണുകൾ നിരീക്ഷിക്കുക

### 🔑 ഡെക്കറേറ്ററുകളിൽ നിന്ന് വ്യത്യാസങ്ങൾ:

| ഫീച്ചർ | ഡെക്കറേറ്റർ | മിഡിൽവെയർ |
|---------|-----------|------------|
| **സ്കോപ്പ്** | ഒറ്റ ഫംഗ്ഷൻ | ഏജന്റിലെ എല്ലാ ഫംഗ്ഷനുകളും |
| **പ്രാചുര്യം** | നിർവചനത്തിൽ നിശ്ചിതം | റൺടൈമിൽ ഡൈനാമിക് |
| **കോൺടെക്സ്റ്റ്** | പരിമിതം | പൂർണ്ണ ഏജന്റ് കോൺടെക്സ്റ്റ് |
| **കമ്പോസിഷൻ** | ഒന്നിലധികം ഡെക്കറേറ്ററുകൾ | മിഡിൽവെയർ പൈപ്പ്‌ലൈൻ |
| **ഏജന്റ്-അവബോധം** | ഇല്ല | ഉണ്ട് (ഏജന്റ് സ്റ്റേറ്റിലേക്ക് ആക്സസ്) |

### 📚 മിഡിൽവെയർ എപ്പോൾ ഉപയോഗിക്കാം:

✅ **മിഡിൽവെയർ ഉപയോഗിക്കുക:**
- ഉപയോക്താവ്/സെഷൻ സ്റ്റേറ്റിനെ അടിസ്ഥാനമാക്കി പെരുമാറ്റം മാറ്റേണ്ടതുണ്ടെങ്കിൽ
- ഒന്നിലധികം ഫംഗ്ഷനുകളിൽ ലജിക് പ്രയോഗിക്കേണ്ടതുണ്ടെങ്കിൽ
- ഏജന്റ്-ലെവൽ കോൺടെക്സ്റ്റിലേക്ക് ആക്സസ് വേണമെങ്കിൽ
- ക്രോസ്-കട്ടിംഗ് ആശങ്കകൾ (ലോഗിംഗ്, ഓത്ത്, മുതലായവ) നടപ്പാക്കുമ്പോൾ

❌ **മിഡിൽവെയർ ഉപയോഗിക്കരുത്:**
- ലളിതമായ ഇൻപുട്ട് വാലിഡേഷൻ (Pydantic ഉപയോഗിക്കുക)
- ഫംഗ്ഷൻ-സ്പെസിഫിക് ലജിക് (ഫംഗ്ഷനിൽ തന്നെ സൂക്ഷിക്കുക)
- ഒറ്റത്തവണ മാറ്റങ്ങൾ (ഫംഗ്ഷൻ മാറ്റുക)

### 🎓 ഉയർന്ന തലത്തിലുള്ള മാതൃകകൾ:

```python
# ഒന്നിലധികം മിഡിൽവെയർ (നിർവഹണ ക്രമം പ്രധാനമാണ്!)
middleware=[
    logging_middleware,      # ആദ്യം ലോഗുകൾ
    auth_middleware,         # ശേഷം ഓത്ത് പരിശോധിക്കുന്നു
    cache_middleware,        # ശേഷം കാഷെ പരിശോധിക്കുന്നു
    rate_limit_middleware,   # ശേഷം നിരക്ക് പരിധി പരിശോധിക്കുന്നു
    priority_check_middleware  # അവസാനം മുൻഗണന പരിശോധിക്കുന്നു
]

# വ്യവസ്ഥാപരമായ മിഡിൽവെയർ നിർവഹണം
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # ഫലത്തിൽ മാറ്റം വരുത്തുക
    else:
        # നിർവഹണം പൂർണ്ണമായും ഒഴിവാക്കുക
        context.result = cached_value
```

### 🔗 ബന്ധപ്പെട്ട ആശയങ്ങൾ:

- **ഏജന്റ് മിഡിൽവെയർ**: agent.run() കോൾസ് തടയുന്നു
- **ഫംഗ്ഷൻ മിഡിൽവെയർ**: ടൂൾ ഫംഗ്ഷൻ കോൾസ് തടയുന്നു (നാം ഉപയോഗിച്ചത്!)
- **മിഡിൽവെയർ പൈപ്പ്‌ലൈൻ**: ക്രമത്തിൽ പ്രവർത്തിക്കുന്ന മിഡിൽവെയറുകളുടെ ശൃംഖല
- **കോൺടെക്സ്റ്റ് പ്രൊപ്പഗേഷൻ**: മിഡിൽവെയർ ശൃംഖലയിലൂടെ സ്റ്റേറ്റ് പാസ്സ് ചെയ്യുക


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖയാണ് വിശ്വസനീയമായ ഉറവിടമായി കണക്കാക്കേണ്ടത്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
